In [1]:
!pip install torch
!pip install ipython
!git clone -b feat/inference/main https://github.com/josephchay/linguify.git

fatal: destination path 'linguify' already exists and is not an empty directory.


In [2]:
%cd linguify
!pip install -r requirements.txt

/content/linguify


In [3]:
import torch
from src.core import Chat
from IPython.display import Audio

In [4]:
chat = Chat()
chat.load_models()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

DVAE_full.pt:   0%|          | 0.00/60.4M [00:00<?, ?B/s]

Decoder.pt:   0%|          | 0.00/104M [00:00<?, ?B/s]

Vocos.pt:   0%|          | 0.00/54.4M [00:00<?, ?B/s]

...

dvae.yaml:   0%|          | 0.00/143 [00:00<?, ?B/s]

gpt.yaml:   0%|          | 0.00/346 [00:00<?, ?B/s]

path.yaml:   0%|          | 0.00/309 [00:00<?, ?B/s]

vocos.yaml:   0%|          | 0.00/460 [00:00<?, ?B/s]

/content/linguify/src/core.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vocos.load_state_dict(torch.load(vocos_ckpt_path))
/content/linguify/src/core.py:77: FutureWa

In [5]:
texts = [
    "Welcome to the advanced healthcare assistant. How may I assist you with your medical inquiries today?",
    "According to the latest diagnostic imaging report, the patient's cardiovascular profile exhibits signs of mild arterial stenosis, necessitating further evaluation through a Doppler ultrasound examination.",
    "The scheduled surgical intervention will involve a minimally invasive laparoscopic cholecystectomy, significantly reducing post-operative recovery time and mitigating potential complications.",
    "Pharmacokinetic analysis indicates that the prescribed medication reaches peak plasma concentration within two hours, with a half-life of approximately eight hours, necessitating bi-daily administration.",
    "The patient's recent hematological panel reveals elevated C-reactive protein levels, suggesting an underlying inflammatory response that warrants further investigation.",
    "Advanced genomic sequencing has identified a potential genetic predisposition to Type 2 Diabetes, highlighting the importance of proactive lifestyle modifications and targeted pharmacotherapy.",
    "The latest research in oncology suggests that immunotherapy, specifically checkpoint inhibitors targeting PD-1 and CTLA-4, is revolutionizing cancer treatment paradigms with remarkable efficacy.",
    "Continuous monitoring via a real-time electrocardiogram has detected intermittent atrial fibrillation episodes, necessitating an in-depth cardiology consultation for risk stratification and management.",
    "The robotic-assisted surgical system enables unparalleled precision in neurosurgical procedures, significantly enhancing patient outcomes and reducing intraoperative risks.",
    "Clinical trials for the next-generation mRNA vaccine have demonstrated a robust immunogenic response, paving the way for enhanced protection against emerging viral variants."
]

In [6]:
wavs = chat.inference(texts, use_decoder=True)

 40%|███▉      | 817/2048 [00:23<00:35, 35.09it/s]


In [7]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [8]:
from src.experimental.llm import LLMApi

client = LLMApi()

Loading model deepseek-ai/deepseek-llm-7b-chat on cuda...


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [9]:
user_question = 'What are the common traditional Chinese medicines still used for treating respiratory illnesses?'
text = client.call(user_question, prompt_version = 'deepseek')
text = client.call(text, prompt_version = 'deepseek_TN')

In [10]:
from huggingface_hub import snapshot_download
import os  # Also needed for os.path.join

download_path = snapshot_download(repo_id='josephchay/LinguifySpeech', allow_patterns=["*.pt"])
spk_stat = torch.load(os.path.join(download_path, 'asset', 'spk_stat.pt'))
rand_spk = torch.randn(768) * spk_stat.chunk(2)[0] + spk_stat.chunk(2)[1]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

<ipython-input-10-47ea9a83793f>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  spk_stat = torch.load(os.path.join(download_path, 'asset', 'spk_stat.pt'))


In [11]:
params_infer_code = {'spk_emb': rand_spk, 'temperature': .3}
params_refine_text = {'prompt': '[oral_2][laugh_0][break_6]'}
# Use the correct method name: inference instead of infer
wav = chat.inference(
    text='Medical advancements in AI-driven diagnostics have significantly improved early cancer detection. Techniques such as liquid biopsy and deep-learning-based imaging analysis enhance accuracy and patient outcomes.',
    params_refine_text=params_refine_text,
    params_infer_code=params_infer_code,
    use_decoder=True
)

 31%|███       | 635/2048 [00:14<00:31, 45.31it/s]


In [12]:
Audio(wav[0], rate=24_000, autoplay=True)